In [ ]:
structure_path = "raw_data/alphafold_structures/"

amino_acid_names = ['ALA', 'ARG', 'ASN', 'ASP', 'ASX', 'CYS', 'GLN', 'MET', 
                    'GLU', 'GLX', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'PHE',
                    'PRO', 'SER', 'THR', 'TRP', 'TYR', 'UNK', 'VAL'
                   ]

In [ ]:
# Creates features for the CNN from psi and phi angles
# inputs:
#  structure:
#    .pdb file downloaded from alphafold (./raw_data/alphafold_structures) https://alphafold.ebi.ac.uk/download
#
#  amino_acid_names:
#    List of 3-letter names e.g., ['ALA', 'ARG', 'ASN', 'ASP', 'ASX', 'CYS', 'GLN', 'MET', ...]
#
# outputs:
#  By default (bin_size=19, bin_size=19, num_amino_acids=23) density maps of psi/phi angles
def make_histogram_example(structure, amino_acid_names, bin_num=19):
    import numpy as np
    import scipy.ndimage
    
    bins = np.linspace(-180, 180, bin_num)
    histograms = np.zeros((len(amino_acid_names), len(bins), len(bins)))  # default (23, 19, 19)

    for r in structure.get_residues():
        if r.internal_coord:
            # Get psi/phi angle. If None, transform to 0
            psi = r.internal_coord.get_angle("psi") or 0
            phi = r.internal_coord.get_angle("phi") or 0

            # np.digitize find the appropriate bin index for a given argument
            # for example, bins = [-180, -120, -60, 0, 60, 120, 180]
            # Given 63, digitize returns 5
            histograms[amino_acid_names.index(r.resname)][np.digitize(psi, bins)][np.digitize(phi, bins)] += 1
            # print(r.internal_coord.get_angle("psi"), r.internal_coord.get_angle("phi"))
    
    # Apply a gaussian filter
    for channel in range(len(amino_acid_names)):
        histograms[channel] = scipy.ndimage.uniform_filter(histograms[channel], size=3, mode="constant")
    
    # Re-order axes to (19, 19, 23) for consistency as CNN inputs have channel at the end
    histograms = np.moveaxis(histograms, 0, -1)
    return histograms

In [ ]:
# Returns list of (uniprot_id, file_name) e.g., [("P49723", "P49723.pdb"), ...]
def get_uniprot_id_and_file_name_tuple_in_path(path):
    from os import listdir
    return {file.split(".")[0]: file
            for file in listdir(path)}  # "raw_data/alphafold_structures"

def make_histograms_dataset(path, label):
    from Bio.PDB import PDBParser
    import pandas as pd
    import numpy as np
    
    df = pd.read_csv(path, sep="\t") # "processed_data/yeast_ess_feat_tsv.tsv"
    pdb_id_and_file_names = get_uniprot_id_and_file_name_tuple_in_path(structure_path)

    parser = PDBParser()
    histograms = list()
    histogram_labels = list()
    histogram_names = list()

    for index, row in df.iterrows():
        uniprot_id = row["#Uniprot_ID"]
        
        if uniprot_id in pdb_id_and_file_names.keys():
            structure = parser.get_structure(uniprot_id, structure_path + pdb_id_and_file_names[uniprot_id])
            structure.atom_to_internal_coordinates()

            hist = make_histogram_example(structure, amino_acid_names)

            histograms.append(hist)
            histogram_names.append(uniprot_id)
            histogram_labels.append(label)  # 0 or 1
            
    histograms = np.asarray(histograms)
    histogram_labels = np.asarray(histogram_labels)
    histogram_names = np.asarray(histogram_names)
    
    return histograms, histogram_labels, histogram_names

In [ ]:
(
    ess_histograms,
    ess_histogram_labels,
    ess_histogram_name
) = make_histograms_dataset("processed_data/yeast_ess_feat_tsv.tsv", 1)

(
    ness_histograms,
    ness_histogram_labels,
    ness_histogram_name
) = make_histograms_dataset("processed_data/yeast_ness_feat_tsv.tsv", 0)

histograms = ess_histograms + ness_histograms
histogram_labels = ess_histogram_labels + ness_histogram_labels
histogram_names = ess_histogram_name + ness_histogram_name

density_map_dict = {
    "data": histograms,
    "labels": histogram_labels,
    "names": histogram_names
}

import pickle
pickle.dump(density_map_dict["data"], open("processed_data/structure_density_maps.pickle", "wb"))
pickle.dump(density_map_dict["names"], open("processed_data/structure_density_maps_names.pickle", "wb"))
pickle.dump(density_map_dict["labels"], open("processed_data/structure_density_maps_labels.pickle", "wb"))